In [7]:

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set_style('whitegrid')

from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
#from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
import pandas as pd
%matplotlib inline
import numpy as np
data_train=pd.read_csv('train.csv')

In [9]:
data_test=pd.read_csv('test.csv')


In [10]:
column_target=['room']
column_train=['wifi1','wifi2','wifi3','wifi4','wifi5','wifi6','wifi7']
X=data_train[column_train]
Y=data_train[column_target]

In [11]:
df_fe=X



In [12]:
# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(df_fe)
# update dataframe 
df_fe = pd.concat((df_fe, pd.DataFrame(ugriz)), axis=1)
df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
df_fe.drop(['wifi1','wifi2','wifi3','wifi4','wifi5','wifi6','wifi7'], axis=1, inplace=True)
df_fe.head()

,PCA_1,PCA_2,PCA_3
0,-13.135324,13.842020,-4.209658
1,-17.066458,13.769786,-0.314761
2,-13.878793,17.501520,0.785219
3,-9.497110,19.992808,4.441609
4,-10.802806,18.093030,3.215021


In [13]:
scaler = MinMaxScaler()
df_fe = scaler.fit_transform(df_fe)

Scaling all values to be within the (0, 1) interval will reduce the distortion due to exceptionally high values and make some algorithms converge faster.

In [14]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_fe,Y,test_size=0.33,random_state=8)



We will split the data into a training and a test part. The models will be trained on the training data set and tested on the test data se

In [15]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression()
log.fit(X_train,y_train)
Y_pred=log.predict(X_test)
acc_lgr=metrics.accuracy_score(y_test,Y_pred)
print(acc_lgr)

0.8868686868686869


/home/saurabh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:

from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)
Y_pred=knn.predict(X_test)
acc_knn=metrics.accuracy_score(y_test,Y_pred)
print(acc_knn)

0.9696969696969697


/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


In [17]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=60,max_depth=10,random_state=7,min_samples_leaf=5)
clf=clf.fit(X_train,y_train)
acc_rfc=metrics.accuracy_score(y_test,Y_pred)
print(acc_rfc)

/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9696969696969697


In [18]:
from sklearn import svm
clf=svm.LinearSVC()
clf.fit(X_train,y_train)
print(clf)
acc_svm=metrics.accuracy_score(y_test,Y_pred)
print(acc_svm)

/home/saurabh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
0.9696969696969697


In [19]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)
Y_pred=gnb.predict(X_test)
acc_gnb=metrics.accuracy_score(y_test,Y_pred)
print(acc_gnb)

0.9777777777777777


/home/saurabh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
results = pd.DataFrame({
    'Model': ['KNN', 'Naive Bayes', 
               'Random Forest', 'SVM','LGR'],
    'Score': [acc_knn, acc_gnb,acc_rfc, acc_svm, acc_lgr],
    })
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Model')
result_df

,Score
Model,
Naive Bayes,0.977778
KNN,0.969697
Random Forest,0.969697
SVM,0.969697
LGR,0.886869


In [21]:
L=data_test[column_train]
dft_fe=L
# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(dft_fe)
# update dataframe 
dft_fe = pd.concat((dft_fe, pd.DataFrame(ugriz)), axis=1)
dft_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
dft_fe.drop(['wifi1','wifi2','wifi3','wifi4','wifi5','wifi6','wifi7'], axis=1, inplace=True)
dft_fe.head()


,PCA_1,PCA_2,PCA_3
0,-6.439799,13.002891,6.182088
1,-13.349704,-12.802221,-0.688184
2,-11.238656,-12.786372,0.606261
3,37.455106,-0.428886,-6.477542
4,-21.000099,15.019399,-5.781194


In [22]:
scaler = MinMaxScaler()
dft_fe = scaler.fit_transform(dft_fe)
gnb.fit(df_fe,Y)
A=gnb.predict(dft_fe)

/home/saurabh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
raw_data={'Room':A}
df = pd.DataFrame(raw_data,data_test['id'],columns=['Room'])
df.to_csv('ans.csv')